### Sistem Rekomendasi Buku dengan Clustering | Collaborative Based

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
buku = pd.read_csv("data/Books.csv",sep= ",",low_memory=False)

In [69]:
buku.head()

ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   
3  http://images.amazon.com/images/P/0374157065.0...   
4  http://images.amazon.com/images/P/0393045218.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   
3  http://images.amazon.com/images/P/0374157065.0...   
4  http://images.amazon.com/images/P/0393045218.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

In [70]:
buku.shape

(271360, 8)

In [71]:
buku.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [72]:
#Menghitung not null dari kolom gambar
totalS = buku['Image-URL-S'].count()
totalM = buku['Image-URL-M'].count()
totalL = buku['Image-URL-L'].count()

print("Jumlah nilai non-null di kolom Image-URL-S:", totalS)
print("Jumlah nilai non-null di kolom Image-URL-M:", totalM)
print("Jumlah nilai non-null di kolom Image-URL-L:", totalL)

Jumlah nilai non-null di kolom Image-URL-S: 271360
Jumlah nilai non-null di kolom Image-URL-M: 271360
Jumlah nilai non-null di kolom Image-URL-L: 271357


In [73]:
#Menyederhanakan tabel buku
buku = buku[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [74]:
buku.head()

ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

In [130]:
buku.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"
}, inplace=True)

In [131]:
buku.head()

ISBN                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  year                   publisher  \
0    Mark P. O. Morford  2002     Oxford University Press   
1  Richard Bruce Wright  2001       HarperFlamingo Canada   
2          Carlo D'Este  1991             HarperPerennial   
3      Gina Bari Kolata  1999        Farrar Straus Giroux   
4       E. J. W. Barber  1999  W. W. Norton &amp; Company   

                                           image_url  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

In [132]:
user = pd.read_csv("data/Users.csv",sep= ",",low_memory=False)

In [133]:
user.head()

User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN

In [134]:
user.shape

(278858, 3)

In [135]:
rating = pd.read_csv("data/Ratings.csv",sep= ",",low_memory=False)

In [136]:
rating.head()

User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6

In [137]:
rating.shape

(1149780, 3)

In [138]:
print(buku.shape)
print(user.shape)
print(rating.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [139]:
rating.rename(columns={
    "User-ID": "user_id",
    "Book-Rating" : "rating"
}, inplace=True)

In [140]:
rating.head()

user_id        ISBN  rating
0   276725  034545104X       0
1   276726  0155061224       5
2   276727  0446520802       0
3   276729  052165615X       3
4   276729  0521795028       6

In [141]:
rating['user_id'].unique().shape

(105283,)

In [142]:
#Mengetahui user yang menilai lebih dari 200 kali
x = rating['user_id'].value_counts()>200
x

11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: user_id, Length: 105283, dtype: bool

In [143]:
x[x].shape

(899,)

In [144]:
y = x[x].index
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [145]:
ratings=rating[rating['user_id'].isin(y)]

In [146]:
ratings.head()

user_id        ISBN  rating
1456   277427  002542730X      10
1457   277427  0026217457       0
1458   277427  003008685X       8
1459   277427  0030615321       0
1460   277427  0060002050       0

In [147]:
ratings.shape

(526356, 3)

In [148]:
#Mengetahui buku dengan rating lebih dari 200 pengguna
rating_join_buku=ratings.merge(buku,on = 'ISBN')

In [149]:
rating_join_buku.shape

(487671, 8)

In [150]:
num_rating = rating_join_buku.groupby('title')['rating'].count().reset_index()

In [151]:
num_rating.head()

title  rating
0   A Light in the Storm: The Civil War Diary of ...       2
1                              Always Have Popsicles       1
2               Apple Magic (The Collector's series)       1
3   Beyond IBM: Leadership Marketing and Finance ...       1
4   Clifford Visita El Hospital (Clifford El Gran...       1

In [152]:
num_rating.rename(columns={"rating":"num_dari_rating"},inplace=True)

In [153]:
num_rating.head()

title  num_dari_rating
0   A Light in the Storm: The Civil War Diary of ...                2
1                              Always Have Popsicles                1
2               Apple Magic (The Collector's series)                1
3   Beyond IBM: Leadership Marketing and Finance ...                1
4   Clifford Visita El Hospital (Clifford El Gran...                1

In [154]:
final_rating = rating_join_buku.merge(num_rating, on='title')

In [155]:
final_rating.head()

user_id        ISBN  rating  \
0   277427  002542730X      10   
1     3363  002542730X       0   
2    11676  002542730X       6   
3    12538  002542730X      10   
4    13552  002542730X       0   

                                               title             author  year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                           image_url  num_dari_rating  
0  http://images.amazon.com/images/P/002542730X.0...               82  
1  http://images.amazon.com/images/P/002542730X.0...               82  
2  http://images.amazon.com/images/P/002542730X.0...               82  
3  http://images.amazon.com/images/P/002542730X.0...               82  
4  http://images.amazon.com/images/P/002542730X.0...               82

In [156]:
final_rating.shape

(487671, 9)

In [157]:
final_rating = final_rating[final_rating['num_dari_rating']>=50]

In [158]:
final_rating.head()

user_id        ISBN  rating  \
0   277427  002542730X      10   
1     3363  002542730X       0   
2    11676  002542730X       6   
3    12538  002542730X      10   
4    13552  002542730X       0   

                                               title             author  year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                           image_url  num_dari_rating  
0  http://images.amazon.com/images/P/002542730X.0...               82  
1  http://images.amazon.com/images/P/002542730X.0...               82  
2  http://images.amazon.com/images/P/002542730X.0...               82  
3  http://images.amazon.com/images/P/002542730X.0...               82  
4  http://images.amazon.com/images/P/002542730X.0...               82

In [159]:
final_rating.shape

(61853, 9)

In [160]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [161]:
final_rating.shape

(59850, 9)

In [162]:
final_rating.head()

user_id        ISBN  rating  \
0   277427  002542730X      10   
1     3363  002542730X       0   
2    11676  002542730X       6   
3    12538  002542730X      10   
4    13552  002542730X       0   

                                               title             author  year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                           image_url  num_dari_rating  
0  http://images.amazon.com/images/P/002542730X.0...               82  
1  http://images.amazon.com/images/P/002542730X.0...               82  
2  http://images.amazon.com/images/P/002542730X.0...               82  
3  http://images.amazon.com/images/P/002542730X.0...               82  
4  http://images.amazon.com/images/P/002542730X.0...               82

In [163]:
buku_pivot = final_rating.pivot_table(columns = 'user_id',index='title',values = 'rating')

In [164]:
buku_pivot

user_id                                             254     2276    2766    \
title                                                                        
1984                                                   9.0     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN    10.0     NaN   
4 Blondes                                              NaN     NaN     NaN   
84 Charing Cross Road                                  NaN     NaN     NaN   
...                                                    ...     ...     ...   
Year of Wonders                                        NaN     NaN     NaN   
You Belong To Me                                       NaN     NaN     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     NaN     NaN     NaN   
Zoya                                                   NaN     NaN     NaN   
\O\" Is for Outlaw"                                    NaN     NaN     NaN   

user_id                                             2977    3363    3757    \
title                                                                        
1984                                                   NaN     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN     NaN     NaN   
4 Blondes                                              NaN     NaN     NaN   
84 Charing Cross Road                                  NaN     NaN     NaN   
...                                                    ...     ...     ...   
Year of Wonders                                        7.0     NaN     NaN   
You Belong To Me                                       NaN     NaN     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     NaN     0.0     NaN   
Zoya                                                   NaN     NaN     NaN   
\O\" Is for Outlaw"                                    NaN     NaN     NaN   

user_id                                             4017    4385    6242    \
title                                                                        
1984                                                   NaN     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN     NaN     NaN   
4 Blondes                                              NaN     NaN     NaN   
84 Charing Cross Road                                  NaN     NaN     NaN   
...                                                    ...     ...     ...   
Year of Wonders                                        NaN     NaN     7.0   
You Belong To Me                                       NaN     NaN     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     NaN     NaN     NaN   
Zoya                                                   NaN     NaN     NaN   
\O\" Is for Outlaw"                                    NaN     NaN     NaN   

user_id                                             6251    ...  274004  \
title                                                       ...           
1984                                                   NaN  ...     NaN   
1st to Die: A Novel                                    NaN  ...     NaN   
2nd Chance                                             NaN  ...     NaN   
4 Blondes                                              0.0  ...     NaN   
84 Charing Cross Road                                  NaN  ...     NaN   
...                                                    ...  ...     ...   
Year of Wonders                                        NaN  ...     NaN   
You Belong To Me                                       NaN  ...     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     0.0  ...     NaN   
Zoya                                                   NaN  ...     NaN   
\O\" Is for Outlaw"                                    

In [165]:
buku_pivot.shape

(742, 888)

In [166]:
buku_pivot.fillna(0,inplace=True)

In [167]:
buku_pivot

user_id                                             254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user_id                                             2977    3363    3757    \
title                                                                        
1984                                                   0.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0     0.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        7.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user_id                                             4017    4385    6242    \
title                                                                        
1984                                                   0.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0     0.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     7.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user_id                                             6251    ...  274004  \
title                                                       ...           
1984                                                   0.0  ...     0.0   
1st to Die: A Novel                                    0.0  ...     0.0   
2nd Chance                                             0.0  ...     0.0   
4 Blondes                                              0.0  ...     0.0   
84 Charing Cross Road                                  0.0  ...     0.0   
...                                                    ...  ...     ...   
Year of Wonders                                        0.0  ...     0.0   
You Belong To Me                                       0.0  ...     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0  ...     0.0   
Zoya                                                   0.0  ...     0.0   
\O\" Is for Outlaw"                                    

In [168]:
from scipy.sparse import csr_matrix

In [169]:
buku_sparse = csr_matrix(buku_pivot)

In [170]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')


In [171]:
model.fit(buku_sparse)

NearestNeighbors(algorithm='brute')

In [172]:
distance, suggestion = model.kneighbors(buku_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors = 6)

In [173]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [174]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [175]:
for i in range(len(suggestion)):
    print(buku_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [176]:
buku_pivot.index[101]

'Carnal Innocence'

In [177]:
nama_buku = buku_pivot.index

In [178]:
nama_buku

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [179]:
import pickle
pickle.dump(model, open('artifacts/model.pkl','wb'))
pickle.dump(nama_buku, open('artifacts/nama_buku.pkl','wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl','wb'))
pickle.dump(buku_pivot, open('artifacts/buku_pivot.pkl','wb'))

In [180]:
def recommend_book(nama_buku):
    buku_id = np.where(buku_pivot.index == nama_buku)[0][0]
    distance, suggestion = model.kneighbors(buku_pivot.iloc[buku_id,:].values.reshape(1,-1),n_neighbors = 6)
    
    for i in range(len(suggestion)):
        buku = buku_pivot.index[suggestion[i]]
        for j in buku: 
            print(j)

In [181]:
namabuku = 'You Belong To Me'
recommend_book(namabuku)

You Belong To Me
The Cradle Will Fall
Exclusive
Loves Music, Loves to Dance
While My Pretty One Sleeps
Before I Say Good-Bye


In [182]:
namabuku2 = 'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values'
recommend_book(namabuku2)

Zen and the Art of Motorcycle Maintenance: An Inquiry into Values
Exclusive
No Safe Place
Jacob Have I Loved
Foucault's Pendulum
The Reef


In [183]:
print(final_rating.columns)

Index(['user_id', 'ISBN', 'rating', 'title', 'author', 'year', 'publisher',
       'image_url', 'num_dari_rating'],
      dtype='object')
